In [ ]:
# Cargar imágenes y etiquetas desde la estructura de carpetas asl_alphabet
import os
import cv2
import numpy as np

ruta_base = './asl_alphabet_train'  # Ajusta si tu ruta es diferente
TAMANO_IMG = 100

clases = sorted([d for d in os.listdir(ruta_base) if os.path.isdir(os.path.join(ruta_base, d))])
train = []
for idx, clase in enumerate(clases):
    ruta_clase = os.path.join(ruta_base, clase)
    for archivo in os.listdir(ruta_clase):
        if archivo.endswith('.jpg') or archivo.endswith('.png'):
            ruta_img = os.path.join(ruta_clase, archivo)
            img = cv2.imread(ruta_img)
            if img is not None:
                img = cv2.resize(img, (TAMANO_IMG, TAMANO_IMG))
                img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                img = img.reshape(TAMANO_IMG, TAMANO_IMG, 1)
                train.append((img, idx))  # Tupla (imagen, etiqueta)

# NO crear X ni y como arrays aquí para evitar duplicación de datos y consumo de RAM
datos = {'train': train}
metadatos = {'clases': clases, 'total_imagenes': len(train), 'tamano_img': TAMANO_IMG}

print(f'Total de imágenes cargadas: {len(train)}')
print(f'Clases encontradas: {clases}')


# Mostrar información básica de los datos y metadatos (simulado)
print('Metadatos:')
print(metadatos)
print('Primer ejemplo:')
ejemplo_img, ejemplo_et = datos['train'][0]
print(f"Clase: {metadatos['clases'][ejemplo_et]}")
print(f"Tamaño imagen: {ejemplo_img.shape}")

# Mostrar 5 ejemplos del set usando datos['train'] y metadatos
import matplotlib.pyplot as plt

fig, axs = plt.subplots(1, 5, figsize=(15, 3))
for i in range(5):
    img, etiqueta = datos['train'][i]
    img = img.reshape(metadatos['tamano_img'], metadatos['tamano_img'])
    axs[i].imshow(img, cmap='gray')
    axs[i].set_title(metadatos['clases'][etiqueta])
    axs[i].axis('off')
plt.show()

# Mostrar un ejemplo de cada clase usando datos['train'] y metadatos
import matplotlib.pyplot as plt

fig, axs = plt.subplots(1, len(metadatos['clases']), figsize=(20, 3))
for idx, clase in enumerate(metadatos['clases']):
    for img, et in datos['train']:
        if et == idx:
            img = img.reshape(metadatos['tamano_img'], metadatos['tamano_img'])
            axs[idx].imshow(img, cmap='gray')
            axs[idx].set_title(clase)
            axs[idx].axis('off')
            break
plt.tight_layout()
plt.show()

#Manipular y visualizar el set
#Lo pasamos a TAMANO_IMG (100x100) y a blanco y negro (solo para visualizar)
import matplotlib.pyplot as plt
import cv2

plt.figure(figsize=(20,20))

TAMANO_IMG=100

for i, (imagen, etiqueta) in enumerate(datos['train'][:25]):
  imagen = imagen.reshape(TAMANO_IMG, TAMANO_IMG)
  plt.subplot(5, 5, i+1)
  plt.xticks([])
  plt.yticks([])
  plt.imshow(imagen, cmap='gray')
  plt.title(metadatos['clases'][etiqueta])
plt.show()

#Variable que contendra todos los pares de los datos (imagen y etiqueta) ya modificados (blanco y negro, 100x100)
datos_entrenamiento = []

for i, (imagen, etiqueta) in enumerate(datos['train']): #Todos los datos
  imagen = cv2.resize(imagen, (TAMANO_IMG, TAMANO_IMG))
  #imagen = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)
  imagen = imagen.reshape(TAMANO_IMG, TAMANO_IMG, 1) #Cambiar tamano a 100,100,1
  datos_entrenamiento.append([imagen, etiqueta])

#Ver los datos del primer indice
datos_entrenamiento[0]

#Ver cuantos datos tengo en la variable
len(datos_entrenamiento)

# Preparar listas X (imagenes) y y (etiquetas) solo para normalización y entrenamiento
X = [imagen for imagen, etiqueta in datos['train']]
y = [etiqueta for imagen, etiqueta in datos['train']]

X

# Normalizar los datos de las X (imagenes) en lotes para evitar problemas de memoria.
import numpy as np

def normalizar_en_lotes(X_lista, tamano_lote=100):
    X_normalizado = []
    for i in range(0, len(X_lista), tamano_lote):
        lote = np.array(X_lista[i:i+tamano_lote]).astype(float) / 255
        X_normalizado.append(lote)
    return np.concatenate(X_normalizado, axis=0)

# Asegurando que X no sea un array antes de la normalización
X = normalizar_en_lotes(X, tamano_lote=100)

# Mostrar la variable y (etiquetas) solo si está definida y es un array
print(y)

# Convertir etiquetas en arreglo simple solo después de la normalización
y = np.array(y)

X.shape

#Crear los modelos iniciales para clasificación multiclase (ASL Alphabet, 29 clases)
# Usan softmax como salida y sparse_categorical_crossentropy como función de pérdida.
num_clases = len(metadatos['clases'])

modeloCNN = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(100, 100, 1)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),

  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(100, activation='relu'),
  tf.keras.layers.Dense(num_clases, activation='softmax')
])

#Compilar modelos para clasificación multiclase (ASL Alphabet, 29 clases)

modeloCNN.compile(optimizer='adam',
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])

from tensorflow.keras.callbacks import TensorBoard

tensorboardCNN = TensorBoard(log_dir='logs/cnn')
modeloCNN.fit(X, y, batch_size=32,
                validation_split=0.15,
                epochs=100,
                callbacks=[tensorboardCNN])

#ver las imagenes de la variable X sin modificaciones por aumento de datos
plt.figure(figsize=(20, 8))
for i in range(10):
  plt.subplot(2, 5, i+1)
  plt.xticks([])
  plt.yticks([])
  plt.imshow(X[i].reshape(100, 100), cmap="gray")

#Realizar el aumento de datos con varias transformaciones. Al final, graficar 10 como ejemplo
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=15,
    zoom_range=[0.7, 1.4],
    horizontal_flip=True,
    vertical_flip=True
)

datagen.fit(X)

plt.figure(figsize=(20,8))

for imagen, etiqueta in datagen.flow(X, y, batch_size=10, shuffle=False):
  for i in range(10):
    plt.subplot(2, 5, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(imagen[i].reshape(100, 100), cmap="gray")
  break

modeloCNN_AD = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(100, 100, 1)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),

  #tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(100, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

modeloCNN_AD.compile(optimizer='adam',
                    loss='binary_crossentropy',
                    metrics=['accuracy'])

#Separar los datos de entrenamiento y los datos de pruebas en variables diferentes

len(X) * .85 #19700
len(X) - 19700 #3562

X_entrenamiento = X[:19700]
X_validacion = X[19700:]

y_entrenamiento = y[:19700]
y_validacion = y[19700:]

#Usar la funcion flow del generador para crear un iterador que podamos enviar como entrenamiento a la funcion FIT del modelo
data_gen_entrenamiento = datagen.flow(X_entrenamiento, y_entrenamiento, batch_size=32)

tensorboardCNN_AD = TensorBoard(log_dir='logs-new/cnn_AD')

modeloCNN_AD.fit(
    data_gen_entrenamiento,
    epochs=150, batch_size=32,
    validation_data=(X_validacion, y_validacion),
    steps_per_epoch=int(np.ceil(len(X_entrenamiento) / float(32))),
    validation_steps=int(np.ceil(len(X_validacion) / float(32))),
    callbacks=[tensorboardCNN_AD]
)

modeloCNN_AD.save('modelo-cnn-ad.h5')

In [ ]:
#Cargar la extension de tensorboard de colab
%load_ext tensorboard

In [ ]:
#Ejecutar tensorboard e indicarle que lea la carpeta "logs"
%tensorboard --logdir logs

In [ ]:
!pip install tensorflowjs

In [ ]:
!mkdir carpeta_salida

In [ ]:
!tensorflowjs_converter --input_format keras perros-gatos-cnn-ad.h5 carpeta_salida